In [ ]:
# !pip install -q torch transformers datasets accelerate scikit-learn seqeval pandas matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving TokenCLS_Dataset.csv to TokenCLS_Dataset.csv


In [ ]:
import os
import pandas as pd
import ast
from datasets import Dataset
from transformers import AutoTokenizer ,AutoModelForTokenClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import torch
import numpy as np

In [ ]:
filename = "TokenCLS_Dataset.csv"
filepath = os.path.abspath(filename)

df = pd.read_csv(filepath)

df["tokens"] = df["tokens"].apply(ast.literal_eval)
df["labels"] = df["labels"].apply(ast.literal_eval)


dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 30000
})

In [ ]:
class BERTTokenClassification:
    def __init__(self, modelname, col_name, num_labels):
        self.modelname = modelname
        self.tokens = col_name
        self.num_labels = num_labels
        self.tokenizer = AutoTokenizer.from_pretrained(modelname)

    def encoding_data(self, dataset, batch_size=2000, batched=True):
        def tokenize(batch):
            tokenized_inputs = self.tokenizer(
                batch["tokens"],
                is_split_into_words=True,
                truncation=True,
                padding='max_length',
                return_attention_mask=True
            )

            labels = []
            for i, label in enumerate(batch["labels"]):
                word_ids = tokenized_inputs.word_ids(batch_index=i)
                label_ids = []
                previous_word_idx = None
                for word_idx in word_ids:
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:
                        label_value = label[word_idx]
                        if 0 <= label_value < self.num_labels:
                            label_ids.append(label_value)
                        else:
                            label_ids.append(-100)
                    else:
                        label_ids.append(-100)
                    previous_word_idx = word_idx
                labels.append(label_ids)

            tokenized_inputs["labels"] = labels
            return tokenized_inputs

        return dataset.map(tokenize, batched=batched, batch_size=batch_size)


    def train(self, train, test, learning_rate, epochs):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = AutoModelForTokenClassification.from_pretrained(self.modelname, num_labels=self.num_labels).to(device)

        training_args = TrainingArguments(
            output_dir='./results',
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate= learning_rate,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            num_train_epochs= epochs,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
        )

        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)

            # Filter out ignored indices, like padding
            mask = labels != -100
            filtered_labels = labels[mask]
            filtered_predictions = predictions[mask]

            return {"accuracy": accuracy_score(filtered_labels, filtered_predictions)}

        self.trainer = Trainer(
            model= model,
            args = training_args,
            train_dataset= train,
            eval_dataset= test,
            compute_metrics= compute_metrics
        )

        self.trainer.train()

    def evaluate(self):
        return self.trainer.evaluate()

    def save_model(self, path):
        self.trainer.save_model(path)
        self.tokenizer.save_pretrained(path)

In [ ]:
bert_token_cls = BERTTokenClassification("distilbert-base-uncased","tokens",7)

tokenized_data = bert_token_cls.encoding_data(dataset, 5000)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
split = tokenized_data.train_test_split(test_size=0.2)
os.environ["WANDB_DISABLED"] = "true"

trained = bert_token_cls.train(split["train"],split['test'],2e-5,3)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000600,0.000309,1.000000
2,0.000300,0.000192,1.000000
3,0.000300,0.000160,1.000000


In [ ]:
result = bert_token_cls.evaluate()
result

{'eval_loss': 0.00016010594845283777,
 'eval_accuracy': 1.0,
 'eval_runtime': 96.3253,
 'eval_samples_per_second': 62.289,
 'eval_steps_per_second': 1.952,
 'epoch': 3.0}

In [ ]:
bert_token_cls.save_model("bert_token_cls_model")
!zip -r bert_token_cls_model.zip bert_token_cls_model
files.download("bert_token_cls_model.zip")

updating: bert_token_cls_model/ (stored 0%)
updating: bert_token_cls_model/config.json (deflated 52%)
updating: bert_token_cls_model/tokenizer.json (deflated 71%)
updating: bert_token_cls_model/training_args.bin (deflated 52%)
updating: bert_token_cls_model/tokenizer_config.json (deflated 75%)
updating: bert_token_cls_model/model.safetensors (deflated 8%)
updating: bert_token_cls_model/vocab.txt (deflated 53%)
updating: bert_token_cls_model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>